In [1]:
import cv2
import mediapipe as mp

In [2]:

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [3]:
import numpy as np
import pandas as pd

In [11]:
import urllib.request
url = 'https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task'
filename = 'face_landmarker_v2_with_blendshapes.task'
urllib.request.urlretrieve(url, filename)

('face_landmarker_v2_with_blendshapes.task',
 <http.client.HTTPMessage at 0x1bbf99b5c40>)

In [4]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [16]:
img = cv2.imread("frame2.jpg",1)


In [20]:
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("frame2.jpg")

# STEP 4: Detect face landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow('image',annotated_image,)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
image

In [ ]:
cv2.destroyAllWindows()
cap.release()

In [4]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame) 
    detection_result = detector.detect(mp_image)
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detection_result)
    cv2.imshow('Raw Webcam Feed', annotated_image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
detection_result.__dict__.keys()

dict_keys(['face_landmarks', 'face_blendshapes', 'facial_transformation_matrixes'])

In [73]:
len(detection_result.face_landmarks[0])
detection_result.face_landmarks[0] [477]   

NormalizedLandmark(x=0.6950324773788452, y=0.20794585347175598, z=0.043302372097969055, visibility=0.0, presence=0.0)

In [ ]:
for id,lm in enumerate(detection_result.face_landmarks[0]):
    x, y,z,v= int(lm.x * iw), int(lm.y * ih),lm.z,lm.visibility
    face_row=[x, y,z,v]
    print(face_row)
        
       

In [ ]:
for row in detection_result.face_landmarks[0]:
    print(row)

In [5]:
import csv
import os

In [6]:
landmarks = ['class']
for val in range(1, 478+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [7]:
with open('coord.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
class_name="alert"

In [12]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame) 
    detection_result = detector.detect(mp_image)
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detection_result)
    try:
        face=detection_result.face_landmarks[0]
        points = list(np.array([[landmark.x, landmark.y, landmark.z,] for landmark in face]).flatten())

        # Append class name
        points.insert(0, class_name)

        # Export to CSV
        with open('coord.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(points)

    except:
        pass
    cv2.imshow('Raw Webcam Feed', annotated_image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [124]:

face=detection_result.face_landmarks[0]

points = list(np.array([[landmark.x, landmark.y, landmark.z,] for landmark in face]).flatten())


In [ ]:
print(points)

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('coord.csv')
X = df.drop('class', axis=1).values # features
y = df['class'] # target value
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
df

In [15]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [17]:
from sklearn.metrics import accuracy_score, recall_score,precision_score # Accuracy metrics 
import pickle 
for algorithm, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algorithm, accuracy_score(y_test, yhat),
          recall_score(y_test, yhat,average='binary',pos_label='drowsy'),
          precision_score(y_test, yhat,average='binary',pos_label='drowsy'))

lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 0.9775280898876404 0.967741935483871 0.989010989010989


In [18]:
with open('drowsiness.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [5]:
import pickle 
with open('drowsiness.pkl', 'rb') as f:
    model1= pickle.load(f)

In [18]:
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame) 
    detection_result = detector.detect(mp_image)
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detection_result)
    try:
        face=detection_result.face_landmarks[0]
        points = list(np.array([[landmark.x, landmark.y, landmark.z,] for landmark in face]).flatten())
        X = pd.DataFrame([points])
        cl = model1.predict(X)[0]
        prob = model1.predict_proba(X)[0]
        cv2.putText(annotated_image, cl, (50,50), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(annotated_image, str(round(prob[np.argmax(prob)],2)), (100,100), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA) 

    except:
        pass
    cv2.imshow('Raw Webcam Feed', annotated_image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [73]:

def process(image_path):    
    base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
    options = vision.FaceLandmarkerOptions(base_options=base_options,
                                           output_face_blendshapes=True,
                                           output_facial_transformation_matrixes=True,
                                           num_faces=1)
    detector = vision.FaceLandmarker.create_from_options(options)
    cap = cv2.VideoCapture(image_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame=frame.copy()
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame) 
            detection_result = detector.detect(mp_image)
            annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detection_result)
            face=detection_result.face_landmarks[0]
            points = list(np.array([[landmark.x, landmark.y, landmark.z,] for landmark in face]).flatten())
            X = pd.DataFrame([points])
            cl = model1.predict(X)[0]
            prob = model1.predict_proba(X)[0]
            cv2.putText(annotated_image, cl, (50,50), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(annotated_image, str(round(prob[np.argmax(prob)],2)), (100,100), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            yield annotated_image



In [8]:
import gradio as gr

In [10]:
cap = cv2.VideoCapture(0)
w=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
record=cv2.VideoWriter('output2.mp4',cv2.VideoWriter_fourcc('D','I','V','X'),20,(int(w),int(h)))
while cap.isOpened():
    ret, frame = cap.read()
    record.write(frame)
    cv2.imshow('Raw Webcam Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
record.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:

interface_video = gr.Interface(
    fn=process,
    inputs=gr.Video(),
    outputs=gr.Image(),
    title="drowsiness detector"
)

In [ ]:
interface_video.queue().launch()

In [78]:
def process2(img):
    cap=cv2.VideoCapture(img)
    while True:
        ret,frame=cap.read()
        if ret:
            return cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
# cap.release()

# cv2.destroyAllWindows()

In [ ]:


def process_frame(frame):
    # Convert the frame to grayscale
    cap=cv2.VideoCapture(frame)
    while True:
        ret,frame=cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Apply a Gaussian blur to the frame
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)

            # Detect edges in the frame using the Canny algorithm
            edges = cv2.Canny(blurred, 50, 150)

            yield edges

# Define the Gradio interface
iface = gr.Interface(fn=process_frame, inputs="video", outputs="image")

# Launch the interface
iface.queue().launch()